In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np

2025-09-12 23:41:51.504776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb.require("core")
wandb.login()

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: Currently logged in as: emmdaz (emmdaz-zzz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(
    project="Experiments Series 1.01",
    config={
        "Layer1": 128,
        "Activation_1": "sigmoid",
        "Dropout1": "No dropout",
        "Layer2": 256,
        "Activation_2": "sigmoid",
        "Dropout2": "No dropout",
        "Layer3": 10,
        "Activation_3": "softmax",
        "Dropout3": "No dropout",
        "Optimizer": "sgd",
        "Metric": "accuracy",
        "Epoch": 30,
        "Batch_size": 20,
        "Eta": 0.001,
        "Loss": "binary_crossentropy"
    }
)

In [4]:
config = wandb.config

In [5]:
ds=mnist.load_data()

In [6]:
(x_train,y_train),(x_test,y_test)=ds
print(x_train.shape,x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [7]:
# Se hace al aplanado de las imágenes
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

# Se modifica el tipo de datos a tipo flotante (para evitar conflictos)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Se normalizan los datos acorde a la intensidad de los píxeles
x_train /= 255.
x_test /= 255.

In [8]:
# Se realiza el one-hot enconding:

# Se definen el número de clases que hay en la base de datos
classes = config.Layer3

# Se realiza la conversión:
y_train = keras.utils.to_categorical(y_train,classes)
y_test = keras.utils.to_categorical(y_test,classes)

In [9]:
# Se configuran los hiperparámetros del modelo:

# Valor del Learning Rate
eta = config.Eta
eta = float(eta) # Tiene que ser un dato de tipo flotante, pues de lo contrario
                 # tiene conflicto

# Número de épocas
epochs = config.Epoch

# Tamaño del mini-batch
mini_batch= config.Batch_size

In [10]:
# Se realiza la creación del modelo a como se había definido usando Numpy:
model = Sequential([
    Dense(config.Layer1, activation = config.Activation_1, input_shape = (784,)),
    Dense(config.Layer2, activation = config.Activation_2),
    Dense(classes, activation = config.Activation_3)
])
model.summary()
# El modelo es secuencial, se van uniendo neurona por neurona de forma consecutiva

/home/emmdzzz/anaconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-12 23:41:56.166465: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2025-09-12 23:41:56.166488: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-09-12 23:41:56.166494: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for ho

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,074 (531.54 KB)

 Trainable params: 136,074 (531.54 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(loss=config.Loss, optimizer = config.Optimizer,
              metrics=[config.Metric])

history = model.fit(x_train, y_train,
                    batch_size = mini_batch,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[WandbMetricsLogger(log_freq=5),
                               WandbModelCheckpoint("models.keras")]
                    )

Epoch 1/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.1255 - loss: 0.3546 - val_accuracy: 0.1136 - val_loss: 0.3228
Epoch 2/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.1556 - loss: 0.3222 - val_accuracy: 0.3050 - val_loss: 0.3202
Epoch 3/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.2665 - loss: 0.3197 - val_accuracy: 0.2478 - val_loss: 0.3174
Epoch 4/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.3194 - loss: 0.3167 - val_accuracy: 0.4481 - val_loss: 0.3138
Epoch 5/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4338 - loss: 0.3129 - val_accuracy: 0.4732 - val_loss: 0.3091
Epoch 6/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4862 - loss: 0.3081 - val_accuracy: 0.5384 - val_loss: 0.3030
Epoch 7/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4967 - loss: 0.3014 - val_accuracy: 0.4949 - val_loss: 0.2949
Epoch 8/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.5308 - loss: 0.2931 - 

In [12]:
wandb.finish()

batch/accuracy,▁▂▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,███████▇▇▇▇▇▇▇▇▆▆▆▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▁▁▁▁
epoch/accuracy,▁▁▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇███████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,████▇▇▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
epoch/val_accuracy,▁▃▂▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇████████
epoch/val_loss,█████▇▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
batch/accuracy,0.83693
